In [1]:
# Basic Analysis and Visualization
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
from datetime import timedelta
# Mapping
import geopandas
import geopy
from geopy.geocoders import Nominatim
import folium
from geopy.extra.rate_limiter import RateLimiter
from folium import plugins
from folium.plugins import MarkerCluster
# Statistical OLS Regression Analysis
%matplotlib inline
import statsmodels.api as sm
from statsmodels.compat import lzip
from statsmodels.formula.api import ols
#Scipy sklearn Predictions
from sklearn.ensemble import GradientBoostingRegressor

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
# Pull in JSON and set index as case number (unique key)
df = pd.read_json("MC.json",orient="records")
df = df.set_index("case")
df

In [ ]:
# Convert time objects
df['occurred'] = pd.to_datetime(df['occurred'])
df['date'] = [d.date() for d in df['occurred']]
df['time'] = [d.time() for d in df['occurred']]
df['day'] = df['occurred'].dt.day_name()
# Find Fractions of Day
df['timeint'] = (df['occurred']-df['occurred'].dt.normalize()).dt.total_seconds()/timedelta(days=1).total_seconds()
# Remove unfounded events
df = df[df['disposition'] != 'Unfounded']

In [ ]:
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('Morgantown, West Virginia')
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.5)
# 2- - create location column
df['location'] = df['address'].apply(geocode)
# 3 - create longitude, latitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns and remove empty points
df = df[df['point'].notnull()]
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head()

In [ ]:
df.groupby("crime")["crime"].count().sort_values()

In [ ]:
VEHICLE ACCIDENT           28
ALARM CONDITION            30
LARCENY                    32
FIRE ALARM                 43
DESTRUCTION OF PROPERTY    55
DRUG INCIDENT              69
TALK WITH OFFICER          74
ABCC VIOLATION             84
ASSIST EMS POLICE          86
BACK TICKET TOW            99

In [ ]:
df.groupby("building")["building"].count().sort_values()
BLUE LOT                                     21
WVU MOUNTAINLAIR                             23
WVU BOREMAN SOUTH                            23
COLLEGE PARK                                 23
WVU SENECA HALL                              25
WVU POLICE DEPARTMENT                        28
WVU SUMMIT HALL                              28
WVU BROOKE TOWER                             29
HEALTH SCIENCE CENTER                        55
WVU DADISMAN HALL                            56

In [ ]:
df.groupby("date")["date"].count().sort_values()
2019-11-01    21 # WVU vs Baylor
2019-10-05    27 # WVU vs Texas
2019-11-09    31 # WVU vs Texas Tech

In [ ]:
# Mark events with names on map
m = folium.Map([39.645,-79.96], zoom_start=14)
for index, row in df.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=3,
                        popup=row['crime'],
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m)
m

In [ ]:
# convert to (n, 2) nd-array format for heatmap
dfmatrix = df[['latitude', 'longitude']].values
# plot heatmap
m.add_child(plugins.HeatMap(dfmatrix, radius=15))
m

In [ ]:
incidents_on_nov09 = df[df['date'] == pd.datetime(2019,11,9).date()]

In [ ]:
# Map points of events
m2 = folium.Map([39.645,-79.96], zoom_start=14)
for index, row in incidents_on_nov09.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=5,
                        popup=row['crime'],
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m2)
# convert to (n, 2) nd-array format for heatmap
dfmatrix = incidents_on_nov09[['latitude', 'longitude']].values
# plot heatmap
m2.add_child(plugins.HeatMap(dfmatrix, radius=15))
m2

In [ ]:
lat = []
long = []
for index, row in incidents_on_nov09.iterrows():
  lat.append(row["latitude"])
  long.append(row["longitude"])
lat1=sum(lat)/len(lat)
lat2=sum(long)/len(long)
folium.CircleMarker([lat1,lat2],
                        radius=5,
                        popup="CENTER LOCATION",
                        color='black',
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m2)
m2

In [ ]:
after9PM = incidents_on_nov09[incidents_on_nov09['time'] >= pd.datetime(2019,11,9,21,0,0).time()]
lat = []
long = []
for index, row in after9PM.iterrows():
  lat.append(row["latitude"])
  long.append(row["longitude"])
lat1=sum(lat)/len(lat)
lat2=sum(long)/len(long)
folium.CircleMarker([lat1,lat2],
                        radius=5,
                        popup="LATE HOURS LOCATION",
                        color='red',
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m2)
m2

In [ ]:
#----------------------------------------------------------------------
#  First the noiseless case
X = np.atleast_2d(df['timeint'].values).T
# Observations
y = np.atleast_2d(df['latitude'].values).T
# Mesh the input space for evaluations of the real function, the prediction and
# its MSE
xx = np.atleast_2d(np.linspace(0, 1, 913)).T
xx = xx.astype(np.float32)
alpha = 0.95
clf = GradientBoostingRegressor(loss='quantile', alpha=alpha,
                                n_estimators=250, max_depth=3,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9)
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_upper = clf.predict(xx)
clf.set_params(alpha=1.0 - alpha)
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_lower = clf.predict(xx)
clf.set_params(loss='ls')
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_pred = clf.predict(xx)
# Plot the function, the prediction and the 90% confidence interval based on
# the MSE
fig = plt.figure()
plt.figure(figsize=(20,10))
plt.plot(X, y, 'b.', markersize=10, label=u'Observations')
plt.plot(xx, y_pred, 'r-', label=u'Prediction')
plt.plot(xx, y_upper, 'k-')
plt.plot(xx, y_lower, 'k-')
plt.fill(np.concatenate([xx, xx[::-1]]),
         np.concatenate([y_upper, y_lower[::-1]]),
         alpha=.5, fc='b', ec='None', label='90% prediction interval')
plt.xlabel('$Time of Day by Fraction$')
plt.ylabel('$Latitude$')
plt.ylim(39.6, 39.7)
plt.legend(loc='upper right')
plt.show()
ypred2 = y_pred

In [ ]:
#----------------------------------------------------------------------
#  First the noiseless case
X = np.atleast_2d(df['timeint'].values).T
# Observations
y = np.atleast_2d(df['longitude'].values).T
# Mesh the input space for evaluations of the real function, the prediction and
# its MSE
xx = np.atleast_2d(np.linspace(0, 1, 913)).T
xx = xx.astype(np.float32)
alpha = 0.95
clf = GradientBoostingRegressor(loss='quantile', alpha=alpha,
                                n_estimators=250, max_depth=3,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9)
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_upper = clf.predict(xx)
clf.set_params(alpha=1.0 - alpha)
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_lower = clf.predict(xx)
clf.set_params(loss='ls')
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_pred = clf.predict(xx)
# Plot the function, the prediction and the 90% confidence interval based on
# the MSE
fig = plt.figure()
plt.figure(figsize=(20,10))
plt.plot(X, y, 'b.', markersize=10, label=u'Observations')
plt.plot(xx, y_pred, 'r-', label=u'Prediction')
plt.plot(xx, y_upper, 'k-')
plt.plot(xx, y_lower, 'k-')
plt.fill(np.concatenate([xx, xx[::-1]]),
         np.concatenate([y_upper, y_lower[::-1]]),
         alpha=.5, fc='b', ec='None', label='90% prediction interval')
plt.xlabel('$Time of Day by Fraction$')
plt.ylabel('$Longitude$')
plt.ylim(-80, -79.9)
plt.legend(loc='upper right')
plt.show()
ypred1 = y_pred

In [ ]:
# Map points of events
m5 = folium.Map([39.645,-79.96], zoom_start=14)
for i in range(len(ypred1)):
    folium.CircleMarker([ypred2[i], ypred1[i]],
                        radius=4,
                        popup=str(i),
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m5)
m5

In [ ]:
dfDowntown = df[df['latitude']<=39.6425]

In [ ]:
#----------------------------------------------------------------------
#  First the noiseless case
X = np.atleast_2d(dfDowntown['timeint'].values).T
# Observations
y = np.atleast_2d(dfDowntown['latitude'].values).T
# Mesh the input space for evaluations of the real function, the prediction and
# its MSE
xx = np.atleast_2d(np.linspace(0, 1, 535)).T
xx = xx.astype(np.float32)
alpha = 0.95
clf = GradientBoostingRegressor(loss='quantile', alpha=alpha,
                                n_estimators=250, max_depth=3,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9)
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_upper = clf.predict(xx)
clf.set_params(alpha=1.0 - alpha)
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_lower = clf.predict(xx)
clf.set_params(loss='ls')
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_pred = clf.predict(xx)
# Plot the function, the prediction and the 90% confidence interval based on
# the MSE
fig = plt.figure()
plt.figure(figsize=(20,10))
plt.plot(X, y, 'b.', markersize=10, label=u'Observations')
plt.plot(xx, y_pred, 'r-', label=u'Prediction')
plt.plot(xx, y_upper, 'k-')
plt.plot(xx, y_lower, 'k-')
plt.fill(np.concatenate([xx, xx[::-1]]),
         np.concatenate([y_upper, y_lower[::-1]]),
         alpha=.5, fc='b', ec='None', label='90% prediction interval')
plt.xlabel('$Time of Day by Fraction$')
plt.ylabel('$Latitude$')
plt.ylim(39.6, 39.7)
plt.legend(loc='upper right')
plt.show()
ypred2 = y_pred

In [ ]:
#----------------------------------------------------------------------
#  First the noiseless case
X = np.atleast_2d(dfDowntown['timeint'].values).T
# Observations
y = np.atleast_2d(dfDowntown['longitude'].values).T
# Mesh the input space for evaluations of the real function, the prediction and
# its MSE
xx = np.atleast_2d(np.linspace(0, 1, 535)).T
xx = xx.astype(np.float32)
alpha = 0.95
clf = GradientBoostingRegressor(loss='quantile', alpha=alpha,
                                n_estimators=250, max_depth=3,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9)
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_upper = clf.predict(xx)
clf.set_params(alpha=1.0 - alpha)
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_lower = clf.predict(xx)
clf.set_params(loss='ls')
clf.fit(X, y)
# Make the prediction on the meshed x-axis
y_pred = clf.predict(xx)
# Plot the function, the prediction and the 90% confidence interval based on
# the MSE
fig = plt.figure()
plt.figure(figsize=(20,10))
plt.plot(X, y, 'b.', markersize=10, label=u'Observations')
plt.plot(xx, y_pred, 'r-', label=u'Prediction')
plt.plot(xx, y_upper, 'k-')
plt.plot(xx, y_lower, 'k-')
plt.fill(np.concatenate([xx, xx[::-1]]),
         np.concatenate([y_upper, y_lower[::-1]]),
         alpha=.5, fc='b', ec='None', label='90% prediction interval')
plt.xlabel('$Time of Day by Fraction$')
plt.ylabel('$Longitude$')
plt.ylim(-80, -79.9)
plt.legend(loc='upper right')
plt.show()
ypred1 = y_pred

In [ ]:
# Map points of events
m7 = folium.Map([39.645,-79.96], zoom_start=14)
for i in range(len(ypred2)):
    folium.CircleMarker([ypred2[i], ypred1[i]],
                        radius=4,
                        popup=str(i),
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m7)
# convert to (n, 2) nd-array format for heatmap
matrix = np.column_stack((ypred2,ypred1))
# plot heatmap
m7.add_child(plugins.HeatMap(matrix, radius=15))
m7

In [ ]:
dfEvansdale = df[df['latitude']>39.6425]